In [25]:
import os
import sys
from urllib.parse import urlparse
from IPython.core.getipython import get_ipython
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append('/Users/juankostelec/Google_drive/Projects/taxGPT-backend')
from scraper.scraper import Scraper
from dotenv import load_dotenv
load_dotenv()    
import openai
import numpy as np
import pandas as pd
import uuid

pd.set_option('display.max_colwidth', None)


In [2]:
# Run the scraper for all the hrefs
METADATA_DIR = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/"
RAW_DATA_DIR = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/testing_complete"
scraper = Scraper(os.path.join(METADATA_DIR, "references.csv"), RAW_DATA_DIR, local=True)
data = scraper.download_all_references()
# scraper.create_downloaded_data_index()


2133it [00:00, 46076.06it/s]


In [44]:
# Now analyze the updated references.csv which files were processed, and which ones were not
# Create a new table for teh saved files:
all_data = scraper.references_data
all_data["is_processed"] = all_data["is_processed"].replace({"True": 1, "False": 0}).astype(float).astype(bool)
# print(all_data.columns)

# Check how many files were not processed at all
not_processed_data = all_data.query("is_processed == False")
not_processed_data['base_url'] = not_processed_data['details_href'].apply(lambda x : urlparse(str(x)).netloc)
# print(not_processed_data["base_url"].value_counts())

# Now look at the data that was not downloaded but was tried to be accessed
processed_data = all_data.query("is_processed == True")
not_downloaded_data = processed_data[processed_data["actual_download_location"].isna()]
not_downloaded_data['base_url'] = not_downloaded_data['details_href'].apply(lambda x : urlparse(str(x)).netloc)
# print(not_downloaded_data["base_url"].value_counts())

# Why was it not downlaoded? (is it a file, a website?)
from utils import is_url_to_file
not_downloaded_data["is_file"] = not_downloaded_data["details_href"].apply(is_url_to_file)
# print(not_downloaded_data["is_file"].value_counts())

# They are all websites:
# How many are such that I actually tried to download??
# fu.gov.si, gov.si, eur-lex.europea.eu, uradni-list.si, oecd.org, fu.gov.si, 
considered_domains = ["eur-lex.europa.eu",
                      "www.fu.gov.si",
                      "www.gov.si",
                      # "ec.europea.eu",
                      # "edavki.durs.si",
                      "www.uradni-list.si",
                      # "www.oecd.org",
                      "fu.gov.si"]
not_downloaded_but_considered_data = not_downloaded_data[not_downloaded_data["base_url"].isin(considered_domains)]
# print(len(not_downloaded_but_considered_data))

# # Wrong location data
# TODO: I need to investigate this in more detail
for idx, row in all_data.iterrows():
    if "32009D0477" in str(row["actual_download_location"]):
        print(row["actual_download_location"])
    if "32009D0477" in str(row["details_href"]):
        print(row["details_href"])
    if "32009D0477" in str(row["used_download_href"]):
        print(row["used_download_href"])
    if "32009D0477" in str(row["actual_download_link"]):
        print(row["actual_download_link"])
    if "32009D0477" in str(row["reference_href_clean"]):
        print(row["reference_href_clean"])
    # if "CELEX".lower() in str(row["actual_download_link"]).lower():
    #     print(row["actual_download_location"], row["used_download_href"]) 


# Check the "Downlaoded files" that have None in their actual_download_location name
downloaded_data = all_data[all_data["actual_download_location"].isna() == False]


     area_name  \
1134    Carina   
1135    Carina   
1136    Carina   
1221    Carina   
1229    Carina   
1260    Carina   
1263    Carina   
1275    Carina   
1330    Carina   
1510    Carina   
1520    Carina   

                                                                                                                                                    area_desc  \
1134  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1135  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1136  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1221  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in 

/var/folders/y1/f5f_tzds45b44fhpyxx38qs40000gn/T/ipykernel_5377/875063542.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_processed_data['base_url'] = not_processed_data['details_href'].apply(lambda x : urlparse(str(x)).netloc)
/var/folders/y1/f5f_tzds45b44fhpyxx38qs40000gn/T/ipykernel_5377/875063542.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_downloaded_data['base_url'] = not_downloaded_data['details_href'].apply(lambda x : urlparse(str(x)).netloc)
/var/folders/y1/f5f_tzds45b44fhpyxx38

In [89]:
schema = [
    "file_id",
    "filename",
    "date_downloaded",
    "area",
    "subarea",
    "section",
    "file_type",
    "raw_filepath",
    "processed_filepath",
    "downloaded_path",
    "file_summary"
]

def get_filetype(path):
    path = str(path)
    if path.endswith(".pdf"):
        return "pdf"
    elif path.endswith(".docx"):
        return "docx"
    elif path.endswith(".doc"):
        return "doc"
    elif path.endswith(".xlsx"):
        return "xlsx"
    elif path.endswith(".xls"):
        return "xls"
    elif path.endswith(".csv"):
        return "csv"
    elif path.endswith(".zip"):
        return "zip"
    elif path.endswith(".html"):
        return "html"
    else:
        print(path)
        return "unknown"


def get_filename(row):
    if row["details_href"] is not None:
        if row["details_href_name"] is not None:
            return row["details_href_name"]
        else:
            return os.path.basename(urlparse(row["details_href"]).path)
    elif row["reference_href_clean"] is not None:
        if row["reference_name"] is not None:
            return row["reference_name"]
        else:
            # Extract the filenmae from the refernce_href_clean
            return os.path.basename(urlparse(row["reference_href_clean"]).path)
    else:
        return None
    

def create_downloaded_data_index():
    new_data = []
    downloaded_data = all_data[all_data["actual_download_location"].isna() == False]
    # downloaded_data = downloaded_data[downloaded_data["actual_download_link"].isna() == False]
    downloaded_data = downloaded_data.where(pd.notnull(downloaded_data), None)

    for _, row in downloaded_data.iterrows():
        file_id = uuid.uuid4()
        href_name = get_filename(row)
        area = row["area_name"]
        subarea = row["reference_name"]
        section = row["details_section"]  # This can be Nan, that is ok
        file_type = get_filetype(row["actual_download_location"])
        raw_url_link = row["actual_download_link"]
        raw_filepath = row["actual_download_location"]
        new_data.append([file_id, href_name, None, area, subarea, section, file_type, raw_url_link,
                         None, raw_filepath, None])
    clean_df = pd.DataFrame(new_data, columns=schema)
    clean_df["file_type"].value_counts()
    clean_df.to_csv(os.path.join(METADATA_DIR, "downloaded_data_index.csv"), index=False)
    return clean_df

clean_df = create_downloaded_data_index()
# Check if I have nan values in the dataset
# print(clean_df.isna().sum())

# Filter to the data that has a nan filename
nan_df = clean_df[clean_df["raw_filepath"].isna()]
# print(nan_df)

# TODO: Figure out why is the filename  (i.e. the reference name) None!!!
# Raw_filepath is a legit file for these files
# Downloaded path sometimes points to nan.pdf/nan.doc files, sometimes to actual files
# Explanation: We download directly from the main FURS website, so we should take the name from the referencE_name, not the details_reference_name
# Some of them are None but without clear reason (we fail to extract the details href name)


# TODO: figure out why the actual_download_link is None in some cases
downloaded_data = all_data[all_data["actual_download_location"].isna() == False]
weird_files = downloaded_data[downloaded_data["actual_download_link"].isna()]
print(weird_files)




     area_name  \
1100    Carina   
1101    Carina   
1134    Carina   
1135    Carina   
1136    Carina   
1152    Carina   
1153    Carina   
1221    Carina   
1229    Carina   
1260    Carina   
1263    Carina   
1275    Carina   
1330    Carina   
1510    Carina   
1520    Carina   

                                                                                                                                                    area_desc  \
1100  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1101  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1134  Informacije o carinskih postopkih in elektronskih sistemih za podporo carinskim postopkom ter prepovedih in omejitvah v potniškem in blagovnem prometu.   
1135  Informacije o carinskih postopkih in elektronskih sistemih za

In [30]:
# Checking round 2 of running full Scraper and what it did not manage to download

new_downloaded_data_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/test_0420/downloaded_data_index.csv"
old_downloaded_data_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/downloaded_data_index.csv"

df_old = pd.read_csv(old_downloaded_data_path)
df_new = pd.read_csv(new_downloaded_data_path)
# Remove the PISRS data from the df_new for fair comparison
df_new["is_pisrs"] = df_new["raw_filepath"].apply(lambda x : "pisrs" in str(x).lower())
df_new = df_new[df_new["is_pisrs"] == False]

print(len(df_old), len(df_new))

# # Check how many files are actually iin the download dir
# new_downloaded_data_dir = os.path.join(os.path.dirname(new_downloaded_data_path), "raw_data")
# old_downloaded_data_dir = os.path.join(os.path.dirname(old_downloaded_data_path), "testing_complete")
# print(len(os.listdir(new_downloaded_data_dir)))
# print(len(os.listdir(old_downloaded_data_dir)))

# Check which files were in the old one but not in the new one
list_filenames = df_new["filename"].unique()

# Filter out from old
df_old_unexplained = df_old[df_old["filename"].isin(list_filenames) == False]


# Let's figure out the original link we tried to access for these files
reference_old_data_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/references.csv"
reference_old = pd.read_csv(reference_old_data_path)
reference_old = reference_old[["details_href", "actual_download_link"]].drop_duplicates()

df_old_unexplained = pd.merge(df_old_unexplained, reference_old,
                              left_on="raw_filepath", right_on="actual_download_link",
                              how="left")
print(df_old_unexplained)



Index(['file_id', 'filename', 'date_downloaded', 'area', 'subarea', 'section',
       'file_type', 'raw_filepath', 'processed_filepath', 'downloaded_path',
       'file_summary'],
      dtype='object')
1012 1012
Empty DataFrame
Columns: [file_id, filename, date_downloaded, area, subarea, section, file_type, raw_filepath, processed_filepath, downloaded_path, file_summary, details_href, actual_download_link]
Index: []
